# Unitree G1 Advanced Use Cases

This notebook covers advanced applications and techniques for the Unitree G1 humanoid robot, including:

1. **Object Manipulation** - Grasping and holding objects (coffee cup example)
2. **Reinforcement Learning** - Training custom behaviors with MimicKit/IsaacGym
3. **Dynamic Balance & Compliance** - Adaptive impedance control
4. **Vision-Based Manipulation** - RealSense integration for object detection
5. **Whole-Body Control** - Coordinated arm-leg motions
6. **Human-Robot Interaction** - Gesture recognition and following
7. **Teleoperation** - VR/Motion capture control interfaces

---

## Prerequisites

This notebook assumes you have completed the basic tutorial (`unitree_g1_sdk_tutorial.ipynb`) and are familiar with:
- High-level locomotion and arm control APIs
- Low-level motor control with PD gains
- Network setup and SDK initialization

---

## Table of Contents

1. [Setup & Dependencies](#setup)
2. [Object Manipulation - Holding a Coffee Cup](#coffee-cup)
3. [Reinforcement Learning with MimicKit](#reinforcement-learning)
4. [Dynamic Balance & Impedance Control](#dynamic-balance)
5. [Vision-Based Grasping](#vision-grasping)
6. [Whole-Body Motion Planning](#whole-body)
7. [Human-Robot Interaction](#hri)
8. [Teleoperation Interfaces](#teleoperation)
9. [Advanced Visualization](#visualization)

---

<a id='setup'></a>
## 1. Setup & Dependencies

Install additional packages for advanced use cases.

In [ ]:
# Install advanced dependencies
!uv pip install torch torchvision  # For neural networks and RL
!uv pip install opencv-python pyrealsense2  # Computer vision
!uv pip install scipy scikit-learn  # Math and ML utilities
!uv pip install gymnasium mujoco  # RL environments (alternative to IsaacGym)
!uv pip install stable-baselines3  # RL algorithms
!uv pip install open3d trimesh  # 3D visualization and processing
!uv pip install mediapipe  # Hand/pose tracking for HRI

In [ ]:
# Standard imports
import time
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Optional

# Computer vision
import cv2
try:
    import pyrealsense2 as rs
except ImportError:
    print("Warning: pyrealsense2 not available")

# Math and control
from scipy.spatial.transform import Rotation
from scipy.interpolate import interp1d
from scipy.optimize import minimize

# Unitree SDK
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelPublisher, ChannelFactoryInitialize
from unitree_sdk2py.idl.unitree_hg.msg.dds_ import LowCmd_, LowState_
from unitree_sdk2py.utils.crc import CRC
from unitree_sdk2py.utils.thread import RecurrentThread
from unitree_sdk2py.g1.loco.g1_loco_client import LocoClient
from unitree_sdk2py.g1.arm.g1_arm_action_client import G1ArmActionClient

print("✓ All imports successful!")

In [ ]:
# Configuration
NETWORK_INTERFACE = "eth0"  # Change to your network interface
ROBOT_DOF = 29  # 23 or 29 DOF variant

# Initialize DDS communication
ChannelFactoryInitialize(0, NETWORK_INTERFACE)
print(f"✓ Initialized communication on {NETWORK_INTERFACE}")

---
<a id='coffee-cup'></a>
## 2. Object Manipulation - Holding a Coffee Cup

This section demonstrates how to grasp and hold a coffee cup using:
- Forward/Inverse kinematics for arm positioning
- Compliant grasp control with force feedback
- Balance compensation during object manipulation

### 2.1 Grasp Pose Planning

We'll define target positions for the gripper (end-effector) and compute joint angles using inverse kinematics.

In [ ]:
class G1ArmKinematics:
    """Simplified kinematics for G1 7-DOF arm (approximation)"""
    
    def __init__(self, arm='left'):
        self.arm = arm
        # Approximate DH parameters (shoulder to wrist)
        # Note: Replace with actual robot parameters from URDF
        self.link_lengths = {
            'shoulder_offset': 0.15,  # Shoulder width from body center
            'upper_arm': 0.35,        # Shoulder to elbow
            'forearm': 0.30,          # Elbow to wrist
            'hand': 0.15              # Wrist to end-effector
        }
    
    def inverse_kinematics_3dof(self, target_pos: np.ndarray) -> Optional[np.ndarray]:
        """
        Simplified IK for shoulder pitch/roll + elbow (3-DOF planar)
        
        Args:
            target_pos: [x, y, z] target position in robot base frame
        
        Returns:
            [shoulder_pitch, shoulder_roll, elbow] joint angles in radians
        """
        x, y, z = target_pos
        
        # Account for shoulder offset
        shoulder_sign = 1 if self.arm == 'left' else -1
        y_arm = y - shoulder_sign * self.link_lengths['shoulder_offset']
        
        # Distance to target in arm plane
        r = np.sqrt(x**2 + y_arm**2 + z**2)
        
        L1 = self.link_lengths['upper_arm']
        L2 = self.link_lengths['forearm'] + self.link_lengths['hand']
        
        # Check reachability
        if r > (L1 + L2) or r < abs(L1 - L2):
            print(f"Warning: Target unreachable (distance: {r:.3f}m)")
            return None
        
        # Elbow angle (using law of cosines)
        cos_elbow = (L1**2 + L2**2 - r**2) / (2 * L1 * L2)
        cos_elbow = np.clip(cos_elbow, -1, 1)
        elbow = np.pi - np.arccos(cos_elbow)  # Elbow down configuration
        
        # Shoulder pitch (vertical plane)
        alpha = np.arctan2(z, np.sqrt(x**2 + y_arm**2))
        beta = np.arctan2(L2 * np.sin(elbow), L1 + L2 * np.cos(elbow))
        shoulder_pitch = alpha - beta
        
        # Shoulder roll (horizontal plane)
        shoulder_roll = np.arctan2(y_arm, x)
        
        return np.array([shoulder_pitch, shoulder_roll, elbow])
    
    def forward_kinematics(self, joint_angles: np.ndarray) -> np.ndarray:
        """
        Compute end-effector position from joint angles
        
        Args:
            joint_angles: [shoulder_pitch, shoulder_roll, shoulder_yaw, elbow, ...]
        
        Returns:
            [x, y, z] end-effector position
        """
        sp, sr = joint_angles[0], joint_angles[1]
        elbow = joint_angles[3] if len(joint_angles) > 3 else 0
        
        L1 = self.link_lengths['upper_arm']
        L2 = self.link_lengths['forearm'] + self.link_lengths['hand']
        
        # Simplified FK (2D plane approximation)
        shoulder_sign = 1 if self.arm == 'left' else -1
        
        x = L1 * np.cos(sp) + L2 * np.cos(sp + elbow)
        y = shoulder_sign * self.link_lengths['shoulder_offset'] + (L1 + L2) * np.sin(sr)
        z = L1 * np.sin(sp) + L2 * np.sin(sp + elbow)
        
        return np.array([x, y, z])

# Test kinematics
left_arm_kin = G1ArmKinematics(arm='left')

# Coffee cup position: 40cm forward, 20cm left, at chest height (80cm)
cup_position = np.array([0.4, 0.2, 0.8])
joint_solution = left_arm_kin.inverse_kinematics_3dof(cup_position)

if joint_solution is not None:
    print(f"Target position: {cup_position}")
    print(f"Joint solution (deg): {np.degrees(joint_solution)}")
    
    # Verify with forward kinematics
    fk_pos = left_arm_kin.forward_kinematics(np.concatenate([joint_solution, [0, 0, 0, 0]]))
    print(f"FK verification: {fk_pos}")
    print(f"Error: {np.linalg.norm(fk_pos - cup_position):.4f}m")

### 2.2 Grasp Execution with Compliant Control

Execute the grasp using low-level control with adaptive impedance.

In [ ]:
class CoffeeGraspController:
    """Controller for grasping and holding a coffee cup"""
    
    def __init__(self, network_interface: str):
        self.network_interface = network_interface
        
        # Create clients
        self.sport_client = LocoClient()
        self.sport_client.SetTimeout(10.0)
        self.sport_client.Init()
        
        # Low-level control setup
        self.lowcmd_publisher = ChannelPublisher("rt/lowcmd", LowCmd_)
        self.lowcmd_publisher.Init()
        
        self.lowstate_subscriber = ChannelSubscriber("rt/lowstate", LowState_)
        self.lowstate_subscriber.Init(self._state_callback, 10)
        
        self.current_state = None
        self.cmd = LowCmd_()
        
        # Grasp parameters
        self.grasp_phases = {
            'approach': {'duration': 2.0, 'kp': 40, 'kd': 2.0},
            'grasp': {'duration': 1.0, 'kp': 30, 'kd': 1.5},
            'hold': {'duration': 5.0, 'kp': 25, 'kd': 1.0},
            'release': {'duration': 1.5, 'kp': 40, 'kd': 2.0}
        }
    
    def _state_callback(self, msg: LowState_):
        """Store current robot state"""
        self.current_state = msg
    
    def grasp_cup(self, cup_position: np.ndarray, hand='left'):
        """
        Execute coffee cup grasp sequence
        
        Args:
            cup_position: [x, y, z] cup location in robot frame
            hand: 'left' or 'right'
        """
        print("🤖 Starting coffee cup grasp sequence...")
        
        # Phase 1: Ensure stable standing posture
        print("  [1/5] Stabilizing base...")
        self.sport_client.StandUp()
        time.sleep(2)
        
        # Phase 2: Pre-grasp pose (arm extended toward cup)
        print("  [2/5] Moving to pre-grasp position...")
        pregrasp_pos = cup_position + np.array([-0.1, 0, 0])  # 10cm before cup
        self._move_arm_to_position(pregrasp_pos, hand, phase='approach')
        
        # Phase 3: Approach cup with compliant motion
        print("  [3/5] Approaching cup...")
        self._move_arm_to_position(cup_position, hand, phase='grasp')
        
        # Phase 4: Close gripper (for dexterous hand) or maintain position
        print("  [4/5] Grasping cup...")
        # Note: G1 may not have gripper - this is placeholder for future hardware
        # For now, maintain wrist position with reduced stiffness
        time.sleep(1)
        
        # Phase 5: Hold with compliant control
        print("  [5/5] Holding cup (compliant mode)...")
        self._hold_position(cup_position, hand, duration=5.0)
        
        print("✓ Grasp sequence complete!")
    
    def _move_arm_to_position(self, target_pos: np.ndarray, hand: str, phase: str):
        """
        Move arm to target position using low-level control
        
        Args:
            target_pos: Target [x, y, z] position
            hand: 'left' or 'right'
            phase: Control phase name (for parameter lookup)
        """
        # Compute IK
        kin = G1ArmKinematics(arm=hand)
        joint_angles = kin.inverse_kinematics_3dof(target_pos)
        
        if joint_angles is None:
            print(f"Warning: Target {target_pos} unreachable")
            return
        
        # Get phase parameters
        params = self.grasp_phases[phase]
        duration = params['duration']
        kp, kd = params['kp'], params['kd']
        
        # Map to motor indices
        if hand == 'left':
            joint_indices = [15, 16, 18]  # LeftShoulderPitch, Roll, Elbow
        else:
            joint_indices = [22, 23, 25]  # RightShoulderPitch, Roll, Elbow
        
        # Execute motion with interpolation
        start_time = time.time()
        rate = 100  # Hz
        
        while time.time() - start_time < duration:
            # Interpolation parameter (0 to 1)
            alpha = (time.time() - start_time) / duration
            
            # Get current positions
            if self.current_state is not None:
                current_q = [self.current_state.motor_state[i].q for i in joint_indices]
            else:
                current_q = [0, 0, 0]
            
            # Smooth interpolation (cubic ease-in-out)
            alpha_smooth = 3*alpha**2 - 2*alpha**3
            
            # Set target positions
            for i, idx in enumerate(joint_indices):
                target_angle = current_q[i] * (1 - alpha_smooth) + joint_angles[i] * alpha_smooth
                self.cmd.motor_cmd[idx].q = target_angle
                self.cmd.motor_cmd[idx].dq = 0
                self.cmd.motor_cmd[idx].kp = kp
                self.cmd.motor_cmd[idx].kd = kd
                self.cmd.motor_cmd[idx].tau = 0
            
            # Publish command
            self.cmd.crc = CRC.Crc(self.cmd)
            self.lowcmd_publisher.Write(self.cmd)
            
            time.sleep(1.0 / rate)
    
    def _hold_position(self, position: np.ndarray, hand: str, duration: float):
        """
        Hold current position with compliant control (low stiffness)
        
        Args:
            position: Target position to maintain
            hand: 'left' or 'right'
            duration: How long to hold (seconds)
        """
        params = self.grasp_phases['hold']
        self._move_arm_to_position(position, hand, 'hold')
        time.sleep(duration)

# Example usage (CAUTION: This will move the robot!)
print("Coffee grasp controller initialized")
print("To execute: controller.grasp_cup(np.array([0.4, 0.2, 0.8]), hand='left')")

# Uncomment to run:
# controller = CoffeeGraspController(NETWORK_INTERFACE)
# controller.grasp_cup(np.array([0.4, 0.2, 0.8]), hand='left')

### 2.3 Grasp Stability Analysis

Monitor contact forces and adjust grasp based on feedback.

In [ ]:
class GraspMonitor:
    """Monitor and analyze grasp stability"""
    
    def __init__(self):
        self.force_history = []
        self.torque_history = []
        self.time_history = []
    
    def analyze_grasp_stability(self, motor_states: List, target_joints: List[int]):
        """
        Analyze grasp stability from motor torque readings
        
        Args:
            motor_states: Current motor state messages
            target_joints: Joint indices involved in grasp
        
        Returns:
            stability_score: 0-1 score (1 = very stable)
        """
        torques = [abs(motor_states[i].tau_est) for i in target_joints]
        
        # Check if torques are within safe range
        torque_variance = np.var(torques)
        mean_torque = np.mean(torques)
        
        # Stability heuristic:
        # - Low variance = stable
        # - Moderate mean torque = good contact
        # - Too high torque = excessive force (slip risk)
        
        variance_score = np.exp(-torque_variance * 10)  # Prefer low variance
        torque_score = np.exp(-abs(mean_torque - 5.0) / 3.0)  # Prefer ~5 Nm
        
        stability = 0.6 * variance_score + 0.4 * torque_score
        
        return stability
    
    def visualize_grasp_forces(self):
        """Plot grasp force history"""
        if len(self.force_history) == 0:
            print("No force data collected yet")
            return
        
        fig, axes = plt.subplots(2, 1, figsize=(12, 8))
        
        # Plot forces
        axes[0].plot(self.time_history, self.force_history, 'b-', linewidth=2)
        axes[0].set_ylabel('Grasp Force (N)', fontsize=11)
        axes[0].set_title('Grasp Force During Hold Phase', fontsize=13)
        axes[0].grid(True, alpha=0.3)
        axes[0].axhline(y=10, color='g', linestyle='--', label='Target Force')
        axes[0].legend()
        
        # Plot torques
        axes[1].plot(self.time_history, self.torque_history, 'r-', linewidth=2)
        axes[1].set_xlabel('Time (s)', fontsize=11)
        axes[1].set_ylabel('Joint Torque (Nm)', fontsize=11)
        axes[1].set_title('Wrist Torque', fontsize=13)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Simulate grasp force data
monitor = GraspMonitor()
t = np.linspace(0, 5, 100)
monitor.time_history = t.tolist()
monitor.force_history = (10 + 2*np.sin(2*np.pi*0.5*t) + np.random.normal(0, 0.5, len(t))).tolist()
monitor.torque_history = (5 + 1*np.cos(2*np.pi*0.3*t) + np.random.normal(0, 0.3, len(t))).tolist()

monitor.visualize_grasp_forces()
print("📊 Grasp force visualization (simulated data)")

---
<a id='reinforcement-learning'></a>
## 3. Reinforcement Learning with MimicKit

Train custom behaviors using imitation learning and reinforcement learning.

### 3.1 Introduction to MimicKit

**MimicKit** is a motion imitation framework for training physics-based controllers. It includes:

- **DeepMimic**: Imitation learning from motion capture data
- **AMP (Adversarial Motion Priors)**: Learning stylized locomotion
- **ASE (Adversarial Skill Embeddings)**: Reusable skill learning
- **ADD**: Physics-based imitation with differential discriminators

**Key Features:**
- Lightweight framework with minimal dependencies
- Integration with NVIDIA IsaacGym for fast parallel simulation
- Supports humanoid and quadruped robots
- Pre-trained models available

**GitHub**: https://github.com/xbpeng/MimicKit

### 3.2 Setting Up MimicKit for G1

#### Installation Steps

```bash
# 1. Install IsaacGym (download from NVIDIA)
# https://developer.nvidia.com/isaac-gym
cd ~/Downloads
tar -xf IsaacGym_Preview_*.tar.gz
cd isaacgym/python
pip install -e .

# 2. Clone and install MimicKit
git clone https://github.com/xbpeng/MimicKit.git
cd MimicKit
pip install -r requirements.txt

# 3. Download assets (motion data, models)
# Follow instructions in MimicKit README
```

#### Creating G1 URDF for IsaacGym

To simulate G1 in IsaacGym, you need a URDF model. If not provided by Unitree:
1. Use CAD files to generate URDF with SolidWorks/Fusion360 plugins
2. Or create simplified URDF manually based on specifications
3. Add collision geometries and inertial properties

In [ ]:
# Check if IsaacGym is available
try:
    from isaacgym import gymapi
    print("✓ IsaacGym available")
    ISAACGYM_AVAILABLE = True
except ImportError:
    print("⚠️  IsaacGym not installed - using Gymnasium/MuJoCo instead")
    ISAACGYM_AVAILABLE = False
    
    # Fallback to Gymnasium
    import gymnasium as gym

### 3.3 Training a Custom Skill with DeepMimic

Example: Training G1 to pick up and hand over an object using motion imitation.

In [ ]:
# Placeholder for MimicKit integration
# This requires IsaacGym and motion capture data

class G1DeepMimicTrainer:
    """
    Wrapper for training G1 behaviors with DeepMimic algorithm
    """
    
    def __init__(self, motion_file: str, urdf_path: str):
        """
        Args:
            motion_file: Path to reference motion data (.npy or .txt)
            urdf_path: Path to G1 URDF file
        """
        self.motion_file = motion_file
        self.urdf_path = urdf_path
        
        # Training hyperparameters (from MimicKit defaults)
        self.config = {
            'learning_rate': 3e-4,
            'batch_size': 4096,
            'num_envs': 4096,  # Parallel simulations
            'horizon': 32,
            'gamma': 0.95,
            'lam': 0.95,
            'motion_weight': 0.5,  # Weight for motion imitation reward
            'task_weight': 0.5,    # Weight for task-specific reward
        }
    
    def load_reference_motion(self):
        """
        Load reference motion data from mocap or keyframes
        
        Motion format: [time, joint_positions, joint_velocities, root_pose]
        """
        # Placeholder - actual implementation depends on data format
        print(f"Loading motion from {self.motion_file}")
        # motion_data = np.load(self.motion_file)
        # return motion_data
        pass
    
    def compute_imitation_reward(self, current_pose, reference_pose):
        """
        Compute reward based on similarity to reference motion
        
        Args:
            current_pose: Current robot joint configuration
            reference_pose: Target pose from reference motion
        
        Returns:
            reward: Scalar reward value
        """
        # Joint position error
        pos_error = np.linalg.norm(current_pose - reference_pose)
        
        # Exponential reward (from DeepMimic paper)
        reward = np.exp(-2.0 * pos_error)
        
        return reward
    
    def train(self, num_iterations: int = 10000):
        """
        Train policy using PPO + motion imitation
        
        Args:
            num_iterations: Number of training iterations
        """
        print("🎓 Starting DeepMimic training...")
        print(f"   Config: {self.config}")
        print(f"   Iterations: {num_iterations}")
        
        # Actual training loop would go here
        # See MimicKit repository for full implementation
        
        print("⚠️  Full training requires MimicKit + IsaacGym installation")
        print("   See: https://github.com/xbpeng/MimicKit")

# Example setup
print("DeepMimic trainer configured")
print("""\nTo train a custom behavior:
1. Record reference motion (mocap or manual keyframes)
2. Convert to MimicKit format (.npy or .txt)
3. Create G1 URDF with proper joint/link definitions
4. Run: trainer = G1DeepMimicTrainer('motion.npy', 'g1.urdf')
5. Execute: trainer.train(num_iterations=10000)
""")

### 3.4 Alternative: Training with Stable-Baselines3 + Gymnasium

If IsaacGym is not available, use Gymnasium with a custom G1 environment.

In [ ]:
# Simplified RL training with Stable-Baselines3
import gymnasium as gym
from gymnasium import spaces
import torch

try:
    from stable_baselines3 import PPO
    from stable_baselines3.common.vec_env import DummyVecEnv
    SB3_AVAILABLE = True
except ImportError:
    print("Install stable-baselines3: pip install stable-baselines3")
    SB3_AVAILABLE = False

class G1ReachingEnv(gym.Env):
    """
    Simple reaching task environment for G1 arm
    
    Goal: Train arm to reach random target positions
    """
    
    def __init__(self):
        super().__init__()
        
        # Action space: 7-DOF arm joint velocities
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(7,), dtype=np.float32
        )
        
        # Observation space: joint positions (7) + target position (3) + distance (1)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32
        )
        
        self.kinematics = G1ArmKinematics('left')
        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # Reset arm to neutral position
        self.joint_positions = np.zeros(7)
        
        # Random target in reachable workspace
        self.target_pos = np.array([
            np.random.uniform(0.2, 0.6),  # x: forward
            np.random.uniform(0.0, 0.4),  # y: left
            np.random.uniform(0.6, 1.2)   # z: height
        ])
        
        self.steps = 0
        
        return self._get_obs(), {}
    
    def _get_obs(self):
        # Current end-effector position
        ee_pos = self.kinematics.forward_kinematics(self.joint_positions)
        distance = np.linalg.norm(ee_pos - self.target_pos)
        
        obs = np.concatenate([
            self.joint_positions,  # 7
            self.target_pos,       # 3
            [distance]             # 1
        ]).astype(np.float32)
        
        return obs
    
    def step(self, action):
        # Apply action (joint velocity commands)
        self.joint_positions += action * 0.1  # Scale velocities
        
        # Clip to joint limits (simplified)
        self.joint_positions = np.clip(self.joint_positions, -np.pi, np.pi)
        
        # Compute reward
        ee_pos = self.kinematics.forward_kinematics(self.joint_positions)
        distance = np.linalg.norm(ee_pos - self.target_pos)
        
        # Reward: negative distance + bonus for reaching
        reward = -distance
        if distance < 0.05:  # Within 5cm
            reward += 10.0
            terminated = True
        else:
            terminated = False
        
        self.steps += 1
        truncated = self.steps >= 100  # Max episode length
        
        return self._get_obs(), reward, terminated, truncated, {}
    
    def render(self):
        pass

if SB3_AVAILABLE:
    # Create and train
    env = G1ReachingEnv()
    
    # Train PPO agent
    print("🎓 Training PPO agent for reaching task...")
    model = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        device='cpu'  # Change to 'cuda' if GPU available
    )
    
    # Train for a few steps (increase for real training)
    print("Training for 10,000 timesteps (increase for better performance)...")
    model.learn(total_timesteps=10000)
    
    # Save model
    model.save("g1_reaching_ppo")
    print("✓ Model saved to g1_reaching_ppo.zip")
    
    # Test trained policy
    print("\nTesting trained policy...")
    obs, _ = env.reset()
    for i in range(100):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            print(f"✓ Reached target in {i+1} steps!")
            break
else:
    print("Install stable-baselines3 to run RL training")

### 3.5 Deploying Trained Policy to Real Robot

Transfer learned behavior from simulation to hardware.

In [ ]:
class PolicyDeployment:
    """
    Deploy trained RL policy to real G1 robot
    """
    
    def __init__(self, model_path: str, network_interface: str):
        # Load trained model
        if SB3_AVAILABLE:
            self.model = PPO.load(model_path)
            print(f"✓ Loaded model from {model_path}")
        
        # Initialize robot interface
        self.network_interface = network_interface
        self.lowstate_subscriber = ChannelSubscriber("rt/lowstate", LowState_)
        self.lowstate_subscriber.Init(self._state_callback, 10)
        
        self.lowcmd_publisher = ChannelPublisher("rt/lowcmd", LowCmd_)
        self.lowcmd_publisher.Init()
        
        self.current_state = None
    
    def _state_callback(self, msg: LowState_):
        self.current_state = msg
    
    def get_observation(self):
        """
        Extract observation from robot state (must match training format)
        """
        if self.current_state is None:
            return None
        
        # Extract joint positions for left arm (indices 15-21)
        joint_positions = np.array([
            self.current_state.motor_state[i].q 
            for i in range(15, 22)
        ])
        
        # Add task-specific observations (e.g., target position)
        # This must match the observation space used during training
        target_pos = np.array([0.4, 0.2, 0.8])  # Example target
        
        # Compute distance
        kin = G1ArmKinematics('left')
        ee_pos = kin.forward_kinematics(joint_positions)
        distance = np.linalg.norm(ee_pos - target_pos)
        
        obs = np.concatenate([joint_positions, target_pos, [distance]])
        return obs.astype(np.float32)
    
    def execute_policy(self, duration: float = 10.0):
        """
        Run trained policy on real robot
        
        Args:
            duration: How long to run (seconds)
        """
        print("🤖 Executing trained policy on robot...")
        print("⚠️  WARNING: Ensure robot is in safe area!")
        
        start_time = time.time()
        rate = 50  # Hz (control frequency)
        
        while time.time() - start_time < duration:
            # Get observation
            obs = self.get_observation()
            if obs is None:
                time.sleep(0.01)
                continue
            
            # Get action from policy
            if SB3_AVAILABLE:
                action, _ = self.model.predict(obs, deterministic=True)
            else:
                action = np.zeros(7)  # Placeholder
            
            # Convert action to motor commands
            cmd = LowCmd_()
            for i, motor_idx in enumerate(range(15, 22)):
                # Action is joint velocity - integrate to position
                current_q = self.current_state.motor_state[motor_idx].q
                target_q = current_q + action[i] * 0.1  # Scale action
                
                cmd.motor_cmd[motor_idx].q = target_q
                cmd.motor_cmd[motor_idx].dq = 0
                cmd.motor_cmd[motor_idx].kp = 40
                cmd.motor_cmd[motor_idx].kd = 2
                cmd.motor_cmd[motor_idx].tau = 0
            
            # Send command
            cmd.crc = CRC.Crc(cmd)
            self.lowcmd_publisher.Write(cmd)
            
            time.sleep(1.0 / rate)
        
        print("✓ Policy execution complete")

print("Policy deployment ready")
print("To deploy: deployer = PolicyDeployment('g1_reaching_ppo.zip', NETWORK_INTERFACE)")
print("           deployer.execute_policy(duration=10.0)")

---
<a id='dynamic-balance'></a>
## 4. Dynamic Balance & Impedance Control

Implement adaptive compliance for robust manipulation during locomotion.

### 4.1 Center of Mass (CoM) Estimation

In [ ]:
class BalanceController:
    """
    Monitor and maintain balance during manipulation tasks
    """
    
    def __init__(self):
        # Robot parameters (approximate for G1)
        self.total_mass = 47.0  # kg (approximate)
        self.foot_separation = 0.20  # m
        
        # Safety thresholds
        self.max_com_offset = 0.08  # meters from support polygon center
        self.max_tilt_angle = 0.15  # radians (~8.5 degrees)
    
    def estimate_com_from_imu(self, imu_data: dict, joint_states: dict) -> np.ndarray:
        """
        Estimate center of mass position from IMU and joint angles
        
        Args:
            imu_data: {'rpy': [roll, pitch, yaw], 'accel': [ax, ay, az]}
            joint_states: Joint positions for all motors
        
        Returns:
            com_position: [x, y, z] in robot base frame
        """
        # Simplified CoM estimation (full version requires link masses)
        # This is a heuristic based on body tilt
        
        roll, pitch, yaw = imu_data['rpy']
        
        # Approximate CoM shift due to tilt
        # Assumes CoM at hip height (~0.8m)
        com_height = 0.8
        
        com_x = com_height * np.tan(pitch)
        com_y = com_height * np.tan(roll)
        com_z = com_height
        
        return np.array([com_x, com_y, com_z])
    
    def check_stability(self, com_pos: np.ndarray, support_polygon: np.ndarray) -> bool:
        """
        Check if CoM is within support polygon
        
        Args:
            com_pos: Center of mass [x, y, z]
            support_polygon: Vertices of support polygon (e.g., foot positions)
        
        Returns:
            is_stable: True if CoM is within polygon
        """
        # Project CoM to ground plane
        com_2d = com_pos[:2]
        
        # For standing with two feet, support polygon is rectangle between feet
        # Simplified: check if CoM is near center (0, 0)
        distance_from_center = np.linalg.norm(com_2d)
        
        is_stable = distance_from_center < self.max_com_offset
        
        return is_stable
    
    def compute_compensatory_motion(self, com_pos: np.ndarray, target_com: np.ndarray) -> dict:
        """
        Compute waist/hip adjustments to shift CoM toward target
        
        Args:
            com_pos: Current CoM position
            target_com: Desired CoM position
        
        Returns:
            joint_adjustments: Dictionary of joint angle corrections
        """
        error = target_com - com_pos
        
        # Simple proportional control
        # Shift waist to compensate for CoM error
        waist_roll_correction = -error[1] * 0.5  # Lateral shift
        waist_pitch_correction = -error[0] * 0.5  # Forward/back shift
        
        adjustments = {
            'WaistRoll': waist_roll_correction,
            'WaistPitch': waist_pitch_correction,
        }
        
        return adjustments

# Test balance controller
balance_ctrl = BalanceController()

# Simulate IMU data with slight tilt
imu_sim = {
    'rpy': [0.05, 0.03, 0.0],  # 5deg roll, 3deg pitch
    'accel': [0, 0, 9.81]
}

com_est = balance_ctrl.estimate_com_from_imu(imu_sim, {})
print(f"Estimated CoM position: {com_est}")

is_stable = balance_ctrl.check_stability(com_est, None)
print(f"Robot stable: {is_stable}")

if not is_stable:
    corrections = balance_ctrl.compute_compensatory_motion(com_est, np.array([0, 0, 0.8]))
    print(f"Balance corrections: {corrections}")

### 4.2 Adaptive Impedance Control

Adjust arm stiffness based on task requirements and contact forces.

In [ ]:
class AdaptiveImpedanceController:
    """
    Dynamically adjust PD gains based on contact forces and task phase
    """
    
    def __init__(self):
        # Define impedance profiles for different tasks
        self.impedance_profiles = {
            'free_motion': {'kp': 60, 'kd': 3.0},     # Stiff for precise positioning
            'contact': {'kp': 20, 'kd': 1.0},          # Compliant for safe interaction
            'grasp': {'kp': 30, 'kd': 1.5},            # Medium for holding
            'dynamic': {'kp': 40, 'kd': 2.5},          # Moderate for moving with object
        }
        
        self.current_profile = 'free_motion'
    
    def detect_contact(self, torque_readings: np.ndarray, threshold: float = 3.0) -> bool:
        """
        Detect contact based on unexpected torques
        
        Args:
            torque_readings: Current motor torques
            threshold: Contact detection threshold (Nm)
        
        Returns:
            in_contact: True if contact detected
        """
        # Check if any torque exceeds threshold
        max_torque = np.max(np.abs(torque_readings))
        return max_torque > threshold
    
    def update_impedance(self, task_phase: str, force_feedback: Optional[np.ndarray] = None) -> dict:
        """
        Update impedance parameters based on task phase and forces
        
        Args:
            task_phase: Current task phase ('free_motion', 'contact', etc.)
            force_feedback: Measured contact forces (optional)
        
        Returns:
            gains: {'kp': ..., 'kd': ...}
        """
        if task_phase in self.impedance_profiles:
            base_gains = self.impedance_profiles[task_phase]
        else:
            base_gains = self.impedance_profiles['free_motion']
        
        gains = base_gains.copy()
        
        # Adapt based on force feedback
        if force_feedback is not None:
            force_magnitude = np.linalg.norm(force_feedback)
            
            # Reduce stiffness if large forces detected (safety)
            if force_magnitude > 10.0:  # N
                gains['kp'] *= 0.7
                gains['kd'] *= 0.8
        
        self.current_profile = task_phase
        return gains
    
    def visualize_impedance_trajectory(self):
        """
        Show how impedance changes during a manipulation task
        """
        # Simulate task sequence
        phases = ['free_motion', 'free_motion', 'contact', 'grasp', 'dynamic', 'dynamic', 'grasp', 'contact', 'free_motion']
        time_per_phase = 1.0  # seconds
        
        times = []
        kp_values = []
        kd_values = []
        phase_labels = []
        
        t = 0
        for phase in phases:
            gains = self.update_impedance(phase)
            times.append(t)
            kp_values.append(gains['kp'])
            kd_values.append(gains['kd'])
            phase_labels.append(phase)
            t += time_per_phase
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
        # Plot Kp
        ax1.step(times, kp_values, 'b-', linewidth=2, where='post', label='Kp')
        ax1.set_ylabel('Position Gain (Kp)', fontsize=11)
        ax1.set_title('Adaptive Impedance During Manipulation Task', fontsize=13)
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Plot Kd
        ax2.step(times, kd_values, 'r-', linewidth=2, where='post', label='Kd')
        ax2.set_xlabel('Time (s)', fontsize=11)
        ax2.set_ylabel('Damping Gain (Kd)', fontsize=11)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # Add phase annotations
        for i, (t, phase) in enumerate(zip(times, phase_labels)):
            if i == 0 or phase != phase_labels[i-1]:
                ax1.axvline(x=t, color='gray', linestyle='--', alpha=0.5)
                ax2.axvline(x=t, color='gray', linestyle='--', alpha=0.5)
                ax1.text(t + 0.1, 55, phase, fontsize=8, rotation=90, alpha=0.7)
        
        plt.tight_layout()
        plt.show()

# Visualize adaptive impedance
impedance_ctrl = AdaptiveImpedanceController()
impedance_ctrl.visualize_impedance_trajectory()
print("📊 Adaptive impedance trajectory")

---
<a id='vision-grasping'></a>
## 5. Vision-Based Grasping

Integrate Intel RealSense depth camera for object detection and grasp planning.

### 5.1 RealSense Camera Setup

In [ ]:
class RealSenseGraspDetector:
    """
    Detect graspable objects using RealSense depth camera
    """
    
    def __init__(self):
        try:
            # Configure RealSense pipeline
            self.pipeline = rs.pipeline()
            config = rs.config()
            config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
            config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
            
            self.pipeline.start(config)
            print("✓ RealSense camera initialized")
            self.camera_available = True
        except Exception as e:
            print(f"⚠️  RealSense not available: {e}")
            self.camera_available = False
    
    def capture_frame(self) -> Tuple[np.ndarray, np.ndarray]:
        """
        Capture RGB and depth frame
        
        Returns:
            color_image: RGB image (H, W, 3)
            depth_image: Depth map (H, W) in millimeters
        """
        if not self.camera_available:
            # Return dummy data
            return np.zeros((480, 640, 3), dtype=np.uint8), np.zeros((480, 640), dtype=np.uint16)
        
        frames = self.pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()
        
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
        
        return color_image, depth_image
    
    def detect_objects(self, color_image: np.ndarray, depth_image: np.ndarray) -> List[dict]:
        """
        Detect objects in the scene
        
        Args:
            color_image: RGB image
            depth_image: Depth map
        
        Returns:
            objects: List of detected objects with [x, y, z, width, height, depth]
        """
        # Simple segmentation: find objects on table using depth
        # Filter depth to table height range (e.g., 600-800mm from camera)
        table_mask = (depth_image > 600) & (depth_image < 800)
        
        # Find connected components (objects)
        contours, _ = cv2.findContours(
            table_mask.astype(np.uint8), 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        objects = []
        for contour in contours:
            if cv2.contourArea(contour) < 500:  # Filter small noise
                continue
            
            # Bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Average depth in region
            roi_depth = depth_image[y:y+h, x:x+w]
            avg_depth = np.median(roi_depth[roi_depth > 0])
            
            objects.append({
                'bbox': (x, y, w, h),
                'depth': avg_depth,
                'center_pixel': (x + w//2, y + h//2)
            })
        
        return objects
    
    def pixel_to_3d(self, pixel_coords: Tuple[int, int], depth: float) -> np.ndarray:
        """
        Convert pixel coordinates to 3D position in robot frame
        
        Args:
            pixel_coords: (u, v) pixel coordinates
            depth: Depth value in millimeters
        
        Returns:
            position_3d: [x, y, z] in robot base frame (meters)
        """
        # Camera intrinsics (example for RealSense D435)
        fx, fy = 616.0, 616.0  # Focal lengths
        cx, cy = 320.0, 240.0  # Principal point
        
        u, v = pixel_coords
        z_cam = depth / 1000.0  # mm to meters
        
        # Camera frame coordinates
        x_cam = (u - cx) * z_cam / fx
        y_cam = (v - cy) * z_cam / fy
        
        # Transform to robot base frame
        # Assuming camera is mounted on head looking forward
        # This transform depends on camera mounting position
        # Example: camera at [0.1, 0, 1.5] looking forward
        camera_offset = np.array([0.1, 0, 1.5])  # Camera position on robot
        
        # Camera frame: x-right, y-down, z-forward
        # Robot frame: x-forward, y-left, z-up
        position_robot = camera_offset + np.array([z_cam, -x_cam, -y_cam])
        
        return position_robot
    
    def visualize_detections(self, color_image: np.ndarray, objects: List[dict]):
        """
        Draw bounding boxes on detected objects
        """
        vis_image = color_image.copy()
        
        for obj in objects:
            x, y, w, h = obj['bbox']
            cv2.rectangle(vis_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Label with depth
            depth_mm = obj['depth']
            label = f"{depth_mm:.0f}mm"
            cv2.putText(vis_image, label, (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Display
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
        plt.title(f"Detected {len(objects)} objects")
        plt.axis('off')
        plt.show()

# Example usage
grasp_detector = RealSenseGraspDetector()

if grasp_detector.camera_available:
    color, depth = grasp_detector.capture_frame()
    objects = grasp_detector.detect_objects(color, depth)
    grasp_detector.visualize_detections(color, objects)
    
    # Compute 3D positions
    for i, obj in enumerate(objects):
        pos_3d = grasp_detector.pixel_to_3d(obj['center_pixel'], obj['depth'])
        print(f"Object {i}: {pos_3d}")
else:
    print("RealSense camera not available - using simulated data")

---
<a id='whole-body'></a>
## 6. Whole-Body Motion Planning

Coordinate legs and arms for complex manipulation during locomotion.

### 6.1 Walking While Carrying Object

In [ ]:
class WholeBodyController:
    """
    Coordinate locomotion and manipulation
    """
    
    def __init__(self, network_interface: str):
        self.sport_client = LocoClient()
        self.sport_client.SetTimeout(10.0)
        self.sport_client.Init()
        
        self.balance_ctrl = BalanceController()
    
    def walk_with_object(self, 
                         target_velocity: Tuple[float, float, float],
                         object_mass: float,
                         duration: float = 5.0):
        """
        Walk while carrying an object, adjusting gait for balance
        
        Args:
            target_velocity: (vx, vy, vyaw) desired velocity
            object_mass: Mass of carried object (kg)
            duration: Walk duration (seconds)
        """
        print(f"🚶 Walking with {object_mass}kg object...")
        
        vx, vy, vyaw = target_velocity
        
        # Adjust velocity based on object mass (conservative scaling)
        mass_factor = max(0.5, 1.0 - object_mass / 10.0)  # Reduce speed if heavy
        vx_adjusted = vx * mass_factor
        vy_adjusted = vy * mass_factor
        vyaw_adjusted = vyaw * mass_factor
        
        print(f"   Adjusted velocity: ({vx_adjusted:.2f}, {vy_adjusted:.2f}, {vyaw_adjusted:.2f})")
        
        # Execute walking motion
        start_time = time.time()
        while time.time() - start_time < duration:
            self.sport_client.Move(vx_adjusted, vy_adjusted, vyaw_adjusted)
            time.sleep(0.1)
        
        # Stop
        self.sport_client.Move(0, 0, 0)
        print("✓ Walking complete")

print("Whole-body controller ready")
print("Example: controller.walk_with_object((0.2, 0, 0), object_mass=1.5, duration=3.0)")

---
<a id='hri'></a>
## 7. Human-Robot Interaction

Detect and respond to human gestures using MediaPipe.

### 7.1 Hand Gesture Recognition

In [ ]:
try:
    import mediapipe as mp
    MEDIAPIPE_AVAILABLE = True
except ImportError:
    print("Install MediaPipe: pip install mediapipe")
    MEDIAPIPE_AVAILABLE = False

if MEDIAPIPE_AVAILABLE:
    class GestureRecognizer:
        """
        Recognize hand gestures for robot control
        """
        
        def __init__(self):
            self.mp_hands = mp.solutions.hands
            self.hands = self.mp_hands.Hands(
                static_image_mode=False,
                max_num_hands=2,
                min_detection_confidence=0.5
            )
            self.mp_draw = mp.solutions.drawing_utils
        
        def detect_gesture(self, image: np.ndarray) -> str:
            """
            Detect gesture from RGB image
            
            Returns:
                gesture_name: 'wave', 'point', 'thumbs_up', 'open_palm', etc.
            """
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = self.hands.process(image_rgb)
            
            if not results.multi_hand_landmarks:
                return 'none'
            
            # Analyze first hand
            hand_landmarks = results.multi_hand_landmarks[0]
            
            # Simple gesture classification based on finger positions
            gesture = self._classify_gesture(hand_landmarks)
            
            return gesture
        
        def _classify_gesture(self, landmarks) -> str:
            """
            Classify gesture based on landmark positions
            """
            # Get fingertip and base positions
            thumb_tip = landmarks.landmark[4]
            index_tip = landmarks.landmark[8]
            middle_tip = landmarks.landmark[12]
            ring_tip = landmarks.landmark[16]
            pinky_tip = landmarks.landmark[20]
            
            wrist = landmarks.landmark[0]
            
            # Check if fingers are extended (simple heuristic)
            index_extended = index_tip.y < wrist.y
            middle_extended = middle_tip.y < wrist.y
            ring_extended = ring_tip.y < wrist.y
            pinky_extended = pinky_tip.y < wrist.y
            
            # Classify
            if all([index_extended, middle_extended, ring_extended, pinky_extended]):
                return 'open_palm'
            elif index_extended and not any([middle_extended, ring_extended, pinky_extended]):
                return 'point'
            else:
                return 'unknown'
    
    print("✓ Gesture recognizer ready")
else:
    print("MediaPipe not available")

---
<a id='teleoperation'></a>
## 8. Teleoperation Interfaces

Control G1 using VR controllers or motion capture.

### 8.1 Keyboard Teleoperation (Simple)

In [ ]:
print("""\n🎮 Teleoperation Interfaces

Advanced teleoperation options:

1. **Keyboard Control** (basic)
   - W/A/S/D: Forward/Left/Back/Right
   - Q/E: Rotate left/right
   - Arrow keys: Arm control

2. **VR Control** (requires VR headset + SteamVR)
   - Use OpenVR/PyOpenVR to read controller poses
   - Map controller positions to arm IK targets
   - Trigger for grasp/release

3. **Motion Capture** (requires mocap system)
   - OptiTrack, Vicon, or marker-less (MediaPipe)
   - Retarget human motions to robot joints
   - Real-time imitation

4. **Joystick/Gamepad** (requires pygame/inputs library)
   - Left stick: Locomotion
   - Right stick: Torso orientation
   - Buttons: Arm gestures

See MimicKit documentation for motion retargeting examples.
""")

---
<a id='visualization'></a>
## 9. Advanced Visualization

### 9.1 3D Robot Visualization with Open3D

In [ ]:
try:
    import open3d as o3d
    OPEN3D_AVAILABLE = True
except ImportError:
    print("Install Open3D: pip install open3d")
    OPEN3D_AVAILABLE = False

if OPEN3D_AVAILABLE:
    print("""\n🎨 3D Visualization with Open3D
    
    To visualize robot state in 3D:
    1. Load robot mesh (STL/OBJ files from CAD)
    2. Update mesh poses based on joint angles (forward kinematics)
    3. Render with Open3D visualizer
    
    Example:
    ```python
    import open3d as o3d
    
    # Load robot meshes
    torso_mesh = o3d.io.read_triangle_mesh('meshes/torso.stl')
    arm_mesh = o3d.io.read_triangle_mesh('meshes/arm.stl')
    
    # Apply transforms based on joint angles
    # (Use forward kinematics)
    
    # Visualize
    o3d.visualization.draw_geometries([torso_mesh, arm_mesh])
    ```
    """)
else:
    print("Open3D not available")

---
## Summary & Next Steps

This notebook covered advanced use cases for the Unitree G1:

✅ **Object Manipulation** - Grasping coffee cup with compliant control  
✅ **Reinforcement Learning** - MimicKit integration for custom behavior training  
✅ **Dynamic Balance** - CoM tracking and adaptive impedance  
✅ **Vision-Based Grasping** - RealSense depth camera for object detection  
✅ **Whole-Body Control** - Coordinated locomotion and manipulation  
✅ **Human-Robot Interaction** - Gesture recognition with MediaPipe  
✅ **Teleoperation** - VR and motion capture interfaces  
✅ **3D Visualization** - Open3D rendering  

### Further Reading

- **MimicKit**: https://github.com/xbpeng/MimicKit
- **IsaacGym**: https://developer.nvidia.com/isaac-gym
- **Stable-Baselines3**: https://stable-baselines3.readthedocs.io/
- **RealSense SDK**: https://github.com/IntelRealSense/librealsense
- **MediaPipe**: https://mediapipe.dev/

---

**⚠️ Safety Notes:**
- Test all advanced behaviors in simulation first
- Use low stiffness gains when testing new motions
- Always have emergency stop ready
- Ensure adequate workspace clearance
- Monitor battery and motor temperatures

---

*Created for Unitree G1 Advanced Applications*  
*Covers RL, vision, manipulation, and HRI*